In [1]:
from flask import Flask, request, jsonify
from twilio. twiml.messaging_response import MessagingResponse
import os
from twilio.rest import Client
import mysql.connector
from mysql.connector import Error
import database as mydb
import pandas as pd 
import textEx
import database
import wanxiang
from flask import Flask, request, send_from_directory
from twilio.twiml.messaging_response import MessagingResponse
import pdfQuoteGenerator
from datetime import datetime

#connect_mysql
try:
    connection = mysql.connector.connect(
        host='quote.c9ac6sewqau0.ap-southeast-2.rds.amazonaws.com',
        database='quote',
        user='admin',
        password='admin123'
    )
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
except Error as e:
    print("Error while connecting to MySQL", e)


#connect EC2 instance
account_sid = 'AC79c2fe6511ac0c9a1c3881c384798e22'
auth_token = 'c9a5e13731974f3a5d548102fbd96629'
client = Client(account_sid, auth_token)

message = client.messages.create(
  from_='whatsapp:+14155238886',
  body='Whatsapp client activated',
  to='whatsapp:+85261520721'
)

print(message.sid)

app = Flask(__name__)

def _formatString(concatText:str,supplier:str,effectiveDate:str) -> tuple:
    
 #(productName, productTag, supplier, category, packing, origin, brand, effectiveDate, spec1, spec2, spec3, spec4, spec5, spec6, price, weightUnit, warehouse, notes)
    brand = textEx.getBrand(concatText)
    country = textEx.getCountry(concatText)
    product = textEx.getProduct(concatText)
    warehouse = textEx.getWarehoue(concatText)
    packing = textEx.getPacking(concatText)
    price = textEx.getPrice(concatText)
    weightUnit = textEx.getWeightUnit(concatText)
    category = textEx.getCategory(concatText)
    spec = textEx.getSpec(concatText)

    # Sort 'spec' in alphabetical order
    spec.sort()

    # Initialize variables with empty strings
    spec1 = spec2 = spec3 = spec4 = spec5 = spec6 = ""

    # Assign values from 'spec' to 'spec1' to 'spec6' based on the length of 'spec'
    if len(spec) > 0:
        spec1 = spec[0]
    if len(spec) > 1:
        spec2 = spec[1]
    if len(spec) > 2:
        spec3 = spec[2]
    if len(spec) > 3:
        spec4 = spec[3]
    if len(spec) > 4:
        spec5 = spec[4]
    if len(spec) > 5:
        spec6 = spec[5]
 

    product_detail = tuple(concatText,product, supplier, category, packing,country,brand, effectiveDate,spec1 , spec2 , spec3 , spec4 , spec5 , spec6,price,weightUnit,warehouse,"")

    return product_detail

def _updateProduct(SQLconnection, productDetail:tuple ):
    database.update_product(SQLconnection,productDetail)
    return None

def is_pdf_coming(request_values) -> bool:
    """
    Determine if the incoming WhatsApp message contains a PDF file.

    Args:
    request_values (Dict): The form data from the incoming request.

    Returns:
    bool: True if a PDF is detected, False otherwise.
    """
    num_media = int(request_values.get('NumMedia', 0))  # Number of media items in the message
    for i in range(num_media):
        media_content_type = request_values.get(f'MediaContentType{i}', '')
        if media_content_type == 'application/pdf':
            return True
    return False

user_states = {}
user_data = {}

@app.route('/pdfs/<filename>')
def serve_pdf(filename):
    # Make sure to validate the filename to avoid security issues
    return send_from_directory('static/pdfs', filename)

@app.route("/wa", methods=['POST'])
def receive_whatsapp_message():
    # Extracting the message SID, sender's number, and message body from the request
    message_sid = request.form.get('MessageSid', '')
    sender = request.form.get('From', '')
    message_body = request.form.get('Body', '')

    print(f"Message SID: {message_sid}, From: {sender}, Message: {message_body}")

    sender = request.form.get('From')
    incoming_msg = request.form.get('Body').strip()
    resp = MessagingResponse()
    msg = resp.message()

    if sender not in user_states:
        # New or reset user interaction
        if incoming_msg == "update":
            msg.body("請提供供應商")
            user_states[sender] = 'awaiting_supplier'
            user_data[sender] = {}
        elif incoming_msg == "quote":
            date_str = "2024-02-26"
            date_datetime = datetime.strptime(date_str, "%Y-%m-%d")

            pdf_path = pdfQuoteGenerator.createQuotation(connection, date_datetime, days=2)

            ngrok_base_url = 'https://ca68-54-153-171-62.ngrok-free.app'  # Update with your actual ngrok URL
            pdf_url = f'{ngrok_base_url}{pdf_path}'
            # Send the PDF URL as a media message
            resp.message("sending PDF")
            resp.message(pdf_url)
            msg.media(pdf_url)
            

        else:
            msg.body("Please type 'update' or 'quote' to start the quotation update process.")
    elif user_states[sender] == 'awaiting_supplier':
        user_data[sender]['supplier'] = incoming_msg  # Store the supplier name
        msg.body("文字報價 還是 PDF報價？")
        user_states[sender] = 'awaiting_quotation_type'
    elif user_states[sender] == 'awaiting_quotation_type':
        if incoming_msg in ["文字報價", "PDF報價"]:
            user_data[sender]['quotation_type'] = incoming_msg
            # Here, you'd normally ask for the actual quotation text or PDF,
            # but for simplicity, let's assume it's the end of the process
            # and we're ready to update the database
            #update_database(user_data[sender])
            msg.body(f"Received {incoming_msg}. Updating the database...")
            # Reset state
            del user_states[sender]
            del user_data[sender]
        else:
            msg.body("Please specify '文字報價' or 'PDF報價'.")
    else:
        # Fallback or unknown state
        msg.body("Sorry, I didn't understand that.")
        # Optionally reset user state here
        if sender in user_states:
            del user_states[sender]
        if sender in user_data:
            del user_data[sender]


    




    ### message recieved
  
    # identify pdf or whatsapp quotation
    
        



    #1 (whatsapp) break down to individual rows

    #2 extract data from whatsapp message
    #formattedData = _formatString(message_str)


    

    #4 ask user if the data formatted is ok
    user_verified = False

    #5 if yes update database

    if user_verified :
        
        for product in formattedData:   
            _updateProduct(connection,formattedData)
            resp.message("Quotation Updated")
    else:
        #6 if no return please check your data again
        resp.message("Please check your data again")


    return str(resp)

if __name__ == '__main__':
    app.run(host="0.0.0.0",debug=False,port=5000)

Connected to MySQL Server version  8.0.35
SMb0bb83c36cf11714c4fa6bb5f99dfd33
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.15:5000
Press CTRL+C to quit


In [9]:
if __name__ == '__main__':
    app.run(host="0.0.0.0",debug=False,port=5000)

SMff918bdd4929ca6d2161e7a30616a13d
